# Combining Datasets 
## MOD14A2 analysis (thermal anomalies)

In [2]:
# Import packages
import os
import re  # regular expressions
import warnings
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

warnings.simplefilter('ignore')

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'nyu', 'rbda_project_data'))


In [3]:
# Create a path to the pre-fire MODIS h4 data
thermal_anomalies_path = os.path.join("thermal_anomalies_11_27_test_2018_2019",
                             "MOD14A2.A2018121.h08v04.006.2018129234132.hdf")

In [4]:
# View dataset metadata
with rio.open(thermal_anomalies_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta


<open DatasetReader name='thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf' mode='r'>


{'driver': 'HDF4',
 'dtype': 'float_',
 'nodata': None,
 'width': 512,
 'height': 512,
 'count': 0,
 'crs': None,
 'transform': Affine(1.0, 0.0, 0.0,
        0.0, 1.0, 0.0)}

In [5]:
# Print all of the subdatasets in the data
with rio.open(thermal_anomalies_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)


HDF4_EOS:EOS_GRID:thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf:MODIS_Grid_8Day_Fire:FireMask
HDF4_EOS:EOS_GRID:thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf:MODIS_Grid_8Day_Fire:QA


In [6]:
# Create list to append arrays (of all type of data)
thermal_anomalies_data = []
thermal_anomalies_names = []

# Open the precipitation HDF5 file 
with rio.open(thermal_anomalies_path) as dataset:
    
    # loop through each subdataset in HDF5 file
    for name in dataset.subdatasets:
        
        # Open the subdataset 
        with rio.open(name) as subdataset:
            modis_meta = subdataset.profile
            
            # Read data as a  2 dimensional array and append to list
            thermal_anomalies_data.append(subdataset.read(1))
            thermal_anomalies_names.append(name);
#             np.savetxt(str(name)[-10:-1] + ".csv", subdataset.read(1), delimiter=",")

            
type(thermal_anomalies_data[0])

numpy.ndarray

In [7]:
thermal_anomalies_data[0]

array([[3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 3],
       ...,
       [3, 3, 3, ..., 5, 5, 5],
       [3, 3, 3, ..., 5, 5, 5],
       [3, 3, 3, ..., 5, 5, 5]], dtype=uint8)

In [8]:
thermal_anomalies_data[0].shape

(1200, 1200)

In [9]:
thermal_anomalies_data[1]

array([[4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       ...,
       [4, 4, 4, ..., 6, 6, 6],
       [4, 4, 4, ..., 6, 6, 6],
       [4, 4, 4, ..., 6, 6, 6]], dtype=uint8)

In [10]:
thermal_anomalies_data[1].shape

(1200, 1200)

## MOD13A2 analysis (vegetation indices)

In [11]:
# Create a path to the pre-fire MODIS h4 data
vegetation_path = os.path.join("vegetation_indices_11_28",
                               "MOD13A2.A2019113.h08v05.006.2019129235845.hdf")


In [12]:
# View dataset metadata
with rio.open(vegetation_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta

<open DatasetReader name='vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf' mode='r'>


{'driver': 'HDF4',
 'dtype': 'float_',
 'nodata': None,
 'width': 512,
 'height': 512,
 'count': 0,
 'crs': None,
 'transform': Affine(1.0, 0.0, 0.0,
        0.0, 1.0, 0.0)}

In [13]:
# Print all of the subdatasets in the data
with rio.open(vegetation_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)


HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days NDVI
HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days relative azimuth angle
HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days composite day of the year
HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days pixel reliability
HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days EVI
HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days VI Quality
HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days red reflectance
HDF4_EOS:EOS_GR

In [14]:
# Create list to append arrays (of all type of data)
vegetation_data = []
vegetation_names = []

# Open the precipitation HDF5 file 
with rio.open(vegetation_path) as dataset:
    
    # loop through each subdataset in HDF5 file
    for name in dataset.subdatasets:
        
        # Open the subdataset 
        with rio.open(name) as subdataset:
            modis_meta = subdataset.profile
            
            # Read data as a  2 dimensional array and append to list
            vegetation_data.append(subdataset.read(1))
            vegetation_names.append(name);
#             np.savetxt(str(name)[-10:-1] + ".csv", subdataset.read(1), delimiter=",")


vegetation_data_stacked = np.stack(vegetation_data)

In [15]:
vegetation_data_stacked

array([[[ -3000,  -3000,  -3000, ...,   2939,   2872,   3401],
        [ -3000,  -3000,  -3000, ...,   3547,   3502,   2666],
        [ -3000,  -3000,  -3000, ...,   3335,   2391,   2264],
        ...,
        [  2472,   2442,   2428, ...,   1564,   1744,   1721],
        [  2276,   2289,   2374, ...,   1795,   1917,   1702],
        [  2345,   2300,   2343, ...,   1714,   1911,   1918]],

       [[ -4000,  -4000,  -4000, ...,  12492,  12440,  12457],
        [ -4000,  -4000,  -4000, ...,  12555,  12563,  12570],
        [ -4000,  -4000,  -4000, ...,  12668,  12673,  12628],
        ...,
        [ -3123,  -3118,  -3119, ...,  12780,  13188,  13422],
        [ -3146,  -3147,  -3148, ...,  14115,  14224,  14125],
        [ -3175,  -3162,  -3163, ...,  15060,  14328,  15004]],

       [[    -1,     -1,     -1, ...,    117,    117,    117],
        [    -1,     -1,     -1, ...,    117,    117,    117],
        [    -1,     -1,     -1, ...,    117,    117,    117],
        ...,
        [   

In [19]:
# ep.plot_bands(vegetation_data_stacked,
#               scale=False)
# plt.show()

In [18]:
# # Plot MODIS RGB
# ep.plot_rgb(vegetation_data_stacked,
#             rgb=[0, 3, 2],
#             title='RGB Image of MODIS Data',
#             stretch=True,
#             figsize=(7, 7))

# plt.show()

In [20]:
vegetation_data[0]

array([[-3000, -3000, -3000, ...,  2939,  2872,  3401],
       [-3000, -3000, -3000, ...,  3547,  3502,  2666],
       [-3000, -3000, -3000, ...,  3335,  2391,  2264],
       ...,
       [ 2472,  2442,  2428, ...,  1564,  1744,  1721],
       [ 2276,  2289,  2374, ...,  1795,  1917,  1702],
       [ 2345,  2300,  2343, ...,  1714,  1911,  1918]], dtype=int16)

In [21]:
vegetation_data[0].shape

(1200, 1200)

In [22]:
thermal_anomalies_names

['HDF4_EOS:EOS_GRID:thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf:MODIS_Grid_8Day_Fire:FireMask',
 'HDF4_EOS:EOS_GRID:thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf:MODIS_Grid_8Day_Fire:QA']

In [23]:
vegetation_names

['HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days NDVI',
 'HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days relative azimuth angle',
 'HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days composite day of the year',
 'HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days pixel reliability',
 'HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days EVI',
 'HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days VI Quality',
 'HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days red r

In [31]:
vegetation_data_stacked[1]

array([[-4000, -4000, -4000, ..., 12492, 12440, 12457],
       [-4000, -4000, -4000, ..., 12555, 12563, 12570],
       [-4000, -4000, -4000, ..., 12668, 12673, 12628],
       ...,
       [-3123, -3118, -3119, ..., 12780, 13188, 13422],
       [-3146, -3147, -3148, ..., 14115, 14224, 14125],
       [-3175, -3162, -3163, ..., 15060, 14328, 15004]], dtype=int32)

## Combining  MOD14A2 and MOD13A2

### Stack the fire mask with the NVDI

In [39]:
# fire_mask_and_nvdi 
fire_mask = thermal_anomalies_data[0]
fire_mask

array([[3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 3],
       ...,
       [3, 3, 3, ..., 5, 5, 5],
       [3, 3, 3, ..., 5, 5, 5],
       [3, 3, 3, ..., 5, 5, 5]], dtype=uint8)

In [40]:
thermal_anomalies_names[0]

'HDF4_EOS:EOS_GRID:thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf:MODIS_Grid_8Day_Fire:FireMask'

In [41]:
vegetation_ndvi = vegetation_data[0]
vegetation_ndvi

array([[-3000, -3000, -3000, ...,  2939,  2872,  3401],
       [-3000, -3000, -3000, ...,  3547,  3502,  2666],
       [-3000, -3000, -3000, ...,  3335,  2391,  2264],
       ...,
       [ 2472,  2442,  2428, ...,  1564,  1744,  1721],
       [ 2276,  2289,  2374, ...,  1795,  1917,  1702],
       [ 2345,  2300,  2343, ...,  1714,  1911,  1918]], dtype=int16)

In [37]:
vegetation_names[0]

'HDF4_EOS:EOS_GRID:vegetation_indices_11_28/MOD13A2.A2019113.h08v05.006.2019129235845.hdf:MODIS_Grid_16DAY_1km_VI:1 km 16 days NDVI'

In [42]:
data_to_stacked = []
data_to_stacked.append(fire_mask)
data_to_stacked.append(vegetation_ndvi)
fire_mask_and_nvdi_stacked = np.stack(data_to_stacked)

In [45]:
fire_mask_and_nvdi_stacked

array([[[    3,     3,     3, ...,     4,     4,     4],
        [    3,     3,     3, ...,     4,     4,     4],
        [    3,     3,     3, ...,     4,     4,     3],
        ...,
        [    3,     3,     3, ...,     5,     5,     5],
        [    3,     3,     3, ...,     5,     5,     5],
        [    3,     3,     3, ...,     5,     5,     5]],

       [[-3000, -3000, -3000, ...,  2939,  2872,  3401],
        [-3000, -3000, -3000, ...,  3547,  3502,  2666],
        [-3000, -3000, -3000, ...,  3335,  2391,  2264],
        ...,
        [ 2472,  2442,  2428, ...,  1564,  1744,  1721],
        [ 2276,  2289,  2374, ...,  1795,  1917,  1702],
        [ 2345,  2300,  2343, ...,  1714,  1911,  1918]]], dtype=int16)

## Save stack of fire mask and ndvi to .csv file (this cannot be done cause it is 3d)

In [47]:
# np.savetxt("fire_mask_and_nvdi_data.csv", fire_mask_and_nvdi_stacked , delimiter=",") 

## Save fire_mask to .csv

In [48]:
np.savetxt("fire_mask.csv", fire_mask, delimiter=",")

## Save ndvi to .csv

In [50]:
np.savetxt("ndvi.csv", vegetation_ndvi, delimiter=",")

# This is a random test to see if github works!!!

In [1]:
vegetation_names[0] 

NameError: name 'vegetation_names' is not defined

In [ ]:
## hopefully it will work!!